In [1]:
import pandas as pd
import numpy as np
import ast
from typing import Dict, List

pd.set_option("display.precision", 6)
pd.set_option("display.max_columns", None)
pd.options.mode.chained_assignment = None

# Предварительная обработка данных

## Предобработка данных о бойцах

### Загрузка данных в датафрейм из csv

In [53]:
fighters_df = pd.read_csv("data/0.fighters_raw.csv", index_col=0)
fighters_df["dateOfBirth"] = pd.to_datetime(fighters_df["dateOfBirth"])
fighters_cols = [
    "id",
    "name",
    "weight",
    "height",
    "armSpan",
    "legSwing",
    "weightCategory.id",
    "weightCategory.name",
    "dateOfBirth",
    "country",
    "city",
    "timezone",
]
fighters_df = fighters_df[fighters_cols]
fighters_df.set_index("id", inplace=True)
fighters_df.head(5)

,name,weight,height,armSpan,legSwing,weightCategory.id,weightCategory.name,dateOfBirth,country,city,timezone
id,,,,,,,,,,,
1,Tanner Boser,115.67,187.96,190.50,NaN,9,Тяжелый вес,1991-08-02,Canada,Bonnyville,America/Edmonton
2,Giacomo Lemos,112.04,190.50,190.50,NaN,9,Тяжелый вес,1989-06-23,Brazil,NaN,America/Sao_Paulo
3,Shamil Abdurakhimov,106.59,190.50,193.04,105.41,9,Тяжелый вес,1981-09-02,Dagestan,Makhachkala,Europe/Moscow
4,Klidson Abreu,92.99,182.88,187.96,NaN,8,Полутяжелый вес,1992-12-24,Brazil,Manaus,America/Manaus
5,Yoshihiro Akiyama,77.11,177.80,190.50,106.68,6,Полусредний вес,1975-07-29,Japan,Ikuno,Asia/Tokyo


### Исправляем поле `country` для бойцов из США
У некоторых бойцов из США в поле `country` указан штат, а не страна.
Также заменяем написание `United States` на `USA`, чтобы название соответствовало данным из таблицы с боями.

In [3]:
usa_state_names = [
    "Alaska",
    "Alabama",
    "Arkansas",
    "American Samoa",
    "Arizona",
    "California",
    "Colorado",
    "Connecticut",
    "District ",
    "of Columbia",
    "Delaware",
    "Florida",
    "Georgia",
    "Guam",
    "Hawaii",
    "Iowa",
    "Idaho",
    "Illinois",
    "Indiana",
    "Kansas",
    "Kentucky",
    "Louisiana",
    "Massachusetts",
    "Maryland",
    "Maine",
    "Michigan",
    "Minnesota",
    "Missouri",
    "Mississippi",
    "Montana",
    "North Carolina",
    "North Dakota",
    "Nebraska",
    "New Hampshire",
    "New Jersey",
    "New Mexico",
    "Nevada",
    "New York",
    "Ohio",
    "Oklahoma",
    "Oregon",
    "Pennsylvania",
    "Puerto Rico",
    "Rhode Island",
    "South Carolina",
    "South Dakota",
    "Tennessee",
    "Texas",
    "Utah",
    "Virginia",
    "Virgin Islands",
    "Vermont",
    "Washington",
    "Wisconsin",
    "West Virginia",
    "Wyoming",
]
fighters_df.loc[fighters_df["country"] == "United States", "country"] = "USA"
fighters_df.loc[fighters_df["country"].isin(usa_state_names), "country"] = "USA"
fighters_df.head(5)

,name,weight,height,armSpan,legSwing,weightCategory.id,weightCategory.name,dateOfBirth,country,city,timezone
id,,,,,,,,,,,
1,Tanner Boser,115.67,187.96,190.50,NaN,9,Тяжелый вес,1991-08-02,Canada,Bonnyville,America/Edmonton
2,Giacomo Lemos,112.04,190.50,190.50,NaN,9,Тяжелый вес,1989-06-23,Brazil,NaN,America/Sao_Paulo
3,Shamil Abdurakhimov,106.59,190.50,193.04,105.41,9,Тяжелый вес,1981-09-02,Dagestan,Makhachkala,Europe/Moscow
4,Klidson Abreu,92.99,182.88,187.96,NaN,8,Полутяжелый вес,1992-12-24,Brazil,Manaus,America/Manaus
5,Yoshihiro Akiyama,77.11,177.80,190.50,106.68,6,Полусредний вес,1975-07-29,Japan,Ikuno,Asia/Tokyo


### Выбросы размаха ног меняем на NaN, для дальнейшей обработки

In [4]:
fighters_df.replace(fighters_df.legSwing.max(), np.nan, inplace=True)
fighters_df.replace(fighters_df.legSwing.min(), np.nan, inplace=True)

### Убираем строки с выбросами роста

In [5]:
fighters_df = fighters_df[fighters_df['height'] < 230]
fighters_df = fighters_df[fighters_df['height'] > 145]

### Убираем строки с выбросами веса

In [7]:
fighters_df = fighters_df[fighters_df['weight'] > 47]
fighters_df = fighters_df[fighters_df['weight'] < 250]

### Находим все возможные весовые категории

In [8]:
avg_weight_in_weight_category = fighters_df.groupby(by="weightCategory.id").mean()['weight']
avg_weight_in_weight_category

weightCategory.id
2      57.063040
3      61.242267
4      64.205865
5      70.417324
6      72.343374
7      80.690843
8      92.152333
9     108.195963
10     52.431429
11     56.808095
12     61.131190
13     63.253333
14     88.915679
Name: weight, dtype: float64

### Дроп строк, где нет веса, роста, размаха рук и размаха ног

In [17]:
fighters_df = fighters_df[~fighters_df['weight'].isna() &
                          ~fighters_df['height'].isna() &
                          ~fighters_df['armSpan'].isna() &
                          ~fighters_df['legSwing'].isna()]

### Замена пустых значений роста на размах рук

In [18]:
def replace_null_height_to_arm_span(row):
    if np.isnan(row['height']) and row['armSpan']:
        arm_span = row['armSpan']
        return arm_span
    return row['height']

fighters_df['height'] = fighters_df.apply(
    lambda row: replace_null_height_to_arm_span(row),
    axis=1
)

### Замена пустых значений размаха рук на рост

In [19]:
def replace_null_arm_span_to_height(row):
    if np.isnan(row['armSpan']) and row['height']:
        height = row['height']
        return height
    return row['armSpan']

fighters_df['armSpan'] = fighters_df.apply(
    lambda row: replace_null_arm_span_to_height(row),
    axis=1
)

### Убираем пустые значения размаха ног, средним по колонке

In [20]:
fighters_df['legSwing'].fillna(np.round(fighters_df['legSwing'].mean(), 1), inplace=True)

## Предобработка данных о боях

### Загружаем данные о боях в датафрейм из csv

In [21]:
events_df = pd.read_csv("data/0.events_raw.csv", index_col=0)
events_df["eventDate.date"] = pd.to_datetime(events_df["eventDate.date"])
events_df.set_index("id", inplace=True)
events_df.head(5)

,avgOdds,city,completed,country,duration,eventDate.date,eventDate.timezone,eventDate.timezone_type,fighterId_1,fighterId_2,fighters,link,name,rounds,timezone,weightCategory.id,weightCategory.name,winMethods,winnerId
id,,,,,,,,,,,,,,,,,,,
5201,[],Denver,True,USA,104.0,1993-11-12,Europe/Berlin,3,1646,1923,"[{'fighterId': 1646, 'fightStats': {'hitsTotal...",http://www.ufcstats.com/fight-details/64139d1d...,UFC 1,1.0,America/Denver,7,Средний вес,['SUB'],1646.0
5202,[],Denver,True,USA,52.0,1993-11-12,Europe/Berlin,3,1777,1883,"[{'fighterId': 1777, 'fightStats': {'hitsTotal...",http://www.ufcstats.com/fight-details/00b07967...,UFC 1,1.0,America/Denver,8,Полутяжелый вес,['SUB'],1777.0
5203,[],Denver,True,USA,59.0,1993-11-12,Europe/Berlin,3,1908,1923,"[{'fighterId': 1908, 'fightStats': {'hitsTotal...",http://www.ufcstats.com/fight-details/ac7ca2ec...,UFC 1,1.0,America/Denver,9,Тяжелый вес,['KO'],1923.0
5204,[],Denver,True,USA,57.0,1993-11-12,Europe/Berlin,3,1631,1646,"[{'fighterId': 1631, 'fightStats': {'hitsTotal...",http://www.ufcstats.com/fight-details/ffd16691...,UFC 1,1.0,America/Denver,8,Полутяжелый вес,['SUB'],1646.0
5205,[],Denver,True,USA,138.0,1993-11-12,Europe/Berlin,3,1646,1924,"[{'fighterId': 1646, 'fightStats': {'hitsTotal...",http://www.ufcstats.com/fight-details/cecdc0da...,UFC 1,1.0,America/Denver,7,Средний вес,['SUB'],1646.0


#### Убираем строки с незавершенными боями и боями, где отсутствует `winnerId`

In [22]:
events_df.drop(events_df[events_df["completed"] == False].index, inplace=True)
events_df.drop(events_df[events_df["winnerId"].isna()].index, inplace=True)

#### В строках, где `winnerId` не совпадает с айди ни одного из бойцов, ставим `winnerId` = 0 (ничья)

In [23]:
events_df.loc[
    (events_df["winnerId"] != events_df["fighterId_1"])
    & (events_df["winnerId"] != events_df["fighterId_2"]),
    "winnerId",
] = 0

#### Удаляем лишние колонки

In [24]:
events_df.drop(
    columns=["completed", "eventDate.timezone", "eventDate.timezone_type", "link"],
    inplace=True,
)
events_df.head(5)

,avgOdds,city,country,duration,eventDate.date,fighterId_1,fighterId_2,fighters,name,rounds,timezone,weightCategory.id,weightCategory.name,winMethods,winnerId
id,,,,,,,,,,,,,,,
5201,[],Denver,USA,104.0,1993-11-12,1646,1923,"[{'fighterId': 1646, 'fightStats': {'hitsTotal...",UFC 1,1.0,America/Denver,7,Средний вес,['SUB'],1646.0
5202,[],Denver,USA,52.0,1993-11-12,1777,1883,"[{'fighterId': 1777, 'fightStats': {'hitsTotal...",UFC 1,1.0,America/Denver,8,Полутяжелый вес,['SUB'],1777.0
5203,[],Denver,USA,59.0,1993-11-12,1908,1923,"[{'fighterId': 1908, 'fightStats': {'hitsTotal...",UFC 1,1.0,America/Denver,9,Тяжелый вес,['KO'],1923.0
5204,[],Denver,USA,57.0,1993-11-12,1631,1646,"[{'fighterId': 1631, 'fightStats': {'hitsTotal...",UFC 1,1.0,America/Denver,8,Полутяжелый вес,['SUB'],1646.0
5205,[],Denver,USA,138.0,1993-11-12,1646,1924,"[{'fighterId': 1646, 'fightStats': {'hitsTotal...",UFC 1,1.0,America/Denver,7,Средний вес,['SUB'],1646.0


### Извлекаем данные из колонок `avgOdds` и `fighters`

#### Парсим колонку `avgOdds`

In [25]:
def parse_odds(row: pd.Series) -> pd.Series:
    """
    Parse 'avgOdds' column.
    :param row: Row of the events dataframe.
    :return: pd.Series with odds for the 1st and the 2nd fighters.
    """
    avg_odds = row["avgOdds"]
    if avg_odds == "[]" or avg_odds == np.nan:
        return pd.Series([np.nan] * 2)
    avg_odds = ast.literal_eval(avg_odds)
    if avg_odds[0]["fighterId"] == row["fighterId_1"]:
        return pd.Series([f.get("value", np.nan) for f in avg_odds])
    else:
        return pd.Series([f.get("value", np.nan) for f in reversed(avg_odds)])

In [26]:
events_df[["f1_odds", "f2_odds"]] = events_df[
    ["avgOdds", "fighterId_1", "fighterId_2"]
].apply(lambda row: parse_odds(row), axis=1)
events_df.drop(columns="avgOdds", inplace=True)
events_df.head(5)

,city,country,duration,eventDate.date,fighterId_1,fighterId_2,fighters,name,rounds,timezone,weightCategory.id,weightCategory.name,winMethods,winnerId,f1_odds,f2_odds
id,,,,,,,,,,,,,,,,
5201,Denver,USA,104.0,1993-11-12,1646,1923,"[{'fighterId': 1646, 'fightStats': {'hitsTotal...",UFC 1,1.0,America/Denver,7,Средний вес,['SUB'],1646.0,NaN,NaN
5202,Denver,USA,52.0,1993-11-12,1777,1883,"[{'fighterId': 1777, 'fightStats': {'hitsTotal...",UFC 1,1.0,America/Denver,8,Полутяжелый вес,['SUB'],1777.0,NaN,NaN
5203,Denver,USA,59.0,1993-11-12,1908,1923,"[{'fighterId': 1908, 'fightStats': {'hitsTotal...",UFC 1,1.0,America/Denver,9,Тяжелый вес,['KO'],1923.0,NaN,NaN
5204,Denver,USA,57.0,1993-11-12,1631,1646,"[{'fighterId': 1631, 'fightStats': {'hitsTotal...",UFC 1,1.0,America/Denver,8,Полутяжелый вес,['SUB'],1646.0,NaN,NaN
5205,Denver,USA,138.0,1993-11-12,1646,1924,"[{'fighterId': 1646, 'fightStats': {'hitsTotal...",UFC 1,1.0,America/Denver,7,Средний вес,['SUB'],1646.0,NaN,NaN


#### Парсим колонку `fighters`

In [27]:
fighter_stats_keys = [
    "hitsTotal",
    "hitsSuccessful",
    "takedownTotal",
    "takedownSuccessful",
    "submissionAttempts",
    "takeovers",
    "accentedHitsTotal",
    "accentedHitsSuccessful",
    "knockdowns",
    "protectionPassage",
    "hitsHeadTotal",
    "hitsHeadSuccessful",
    "hitsBodyTotal",
    "hitsBodySuccessful",
    "hitsLegsTotal",
    "hitsLegsSuccessful",
    "accentedHitsPositionDistanceTotal",
    "accentedHitsPositionDistanceSuccessful",
    "accentedHitsPositionClinchTotal",
    "accentedHitsPositionClinchSuccessful",
    "accentedHitsPositionParterTotal",
    "accentedHitsPositionParterSuccessful",
]


def get_fighter_stats_cols() -> List[str]:
    """
    Get list of fight stats column names for each fighter.
    :return: List of column names with 'f1_' prefix
    for the first fighter and 'f2_' prefix for the second.
    """
    fighter_stats_cols = []
    for i in range(1, 3):
        for k in fighter_stats_keys:
            fighter_stats_cols.append(f"f{i}_{k}")
    return fighter_stats_cols


def sum_round_stats(stats: List[Dict[str, int]]) -> List[int]:
    """
    Sum stats for a fighter for all rounds of a fight.
    :param stats: List with stats from object of 'fighters' column.
    :return: Stats for all rounds for a fighter as a list.
    """
    if len(stats) == 0:
        return [np.nan for _ in range(len(fighter_stats_keys))]
    res = {k: 0 for k in fighter_stats_keys}
    for i in stats:
        for k in res:
            res[k] = i.get(k, 0)
    return list(res.values())


def parse_fight_data(row: pd.Series) -> pd.Series:
    """
    Parse 'fighters' column.
    :param row: Row of the events dataframe.
    :return: pd.Series with stats for both fighters.
    """
    fighters = row["fighters"]
    if fighters == "[]" or fighters == np.nan:
        return pd.Series([np.nan for _ in range(len(fighter_stats_keys))])
    cols = []
    fighters = ast.literal_eval(fighters)
    if fighters[0]["fighterId"] == row["fighterId_2"]:
        fighters = reversed(fighters)
    for f in fighters:
        cols.extend(sum_round_stats(f["roundStats"]))
    return pd.Series(cols)

In [28]:
events_df[get_fighter_stats_cols()] = events_df[
    ["fighters", "fighterId_1", "fighterId_2"]
].apply(lambda row: parse_fight_data(row), axis=1)
events_df.drop(columns="fighters", inplace=True)
events_df.head(5)

,city,country,duration,eventDate.date,fighterId_1,fighterId_2,name,rounds,timezone,weightCategory.id,weightCategory.name,winMethods,winnerId,f1_odds,f2_odds,f1_hitsTotal,f1_hitsSuccessful,f1_takedownTotal,f1_takedownSuccessful,f1_submissionAttempts,f1_takeovers,f1_accentedHitsTotal,f1_accentedHitsSuccessful,f1_knockdowns,f1_protectionPassage,f1_hitsHeadTotal,f1_hitsHeadSuccessful,f1_hitsBodyTotal,f1_hitsBodySuccessful,f1_hitsLegsTotal,f1_hitsLegsSuccessful,f1_accentedHitsPositionDistanceTotal,f1_accentedHitsPositionDistanceSuccessful,f1_accentedHitsPositionClinchTotal,f1_accentedHitsPositionClinchSuccessful,f1_accentedHitsPositionParterTotal,f1_accentedHitsPositionParterSuccessful,f2_hitsTotal,f2_hitsSuccessful,f2_takedownTotal,f2_takedownSuccessful,f2_submissionAttempts,f2_takeovers,f2_accentedHitsTotal,f2_accentedHitsSuccessful,f2_knockdowns,f2_protectionPassage,f2_hitsHeadTotal,f2_hitsHeadSuccessful,f2_hitsBodyTotal,f2_hitsBodySuccessful,f2_hitsLegsTotal,f2_hitsLegsSuccessful,f2_accentedHitsPositionDistanceTotal,f2_accentedHitsPositionDistanceSuccessful,f2_accentedHitsPositionClinchTotal,f2_accentedHitsPositionClinchSuccessful,f2_accentedHitsPositionParterTotal,f2_accentedHitsPositionParterSuccessful
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5201,Denver,USA,104.0,1993-11-12,1646,1923,UFC 1,1.0,America/Denver,7,Средний вес,['SUB'],1646.0,NaN,NaN,4.0,3.0,3.0,1.0,1.0,0.0,2.0,1.0,0.0,2.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5202,Denver,USA,52.0,1993-11-12,1777,1883,UFC 1,1.0,America/Denver,8,Полутяжелый вес,['SUB'],1777.0,NaN,NaN,9.0,3.0,2.0,1.0,1.0,0.0,9.0,3.0,0.0,1.0,7.0,3.0,1.0,0.0,1.0,0.0,9.0,3.0,0.0,0.0,0.0,0.0,7.0,1.0,0.0,0.0,0.0,0.0,7.0,1.0,0.0,0.0,5.0,0.0,1.0,0.0,1.0,1.0,7.0,1.0,0.0,0.0,0.0,0.0
5203,Denver,USA,59.0,1993-11-12,1908,1923,UFC 1,1.0,America/Denver,9,Тяжелый вес,['KO'],1923.0,NaN,NaN,3.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,17.0,11.0,0.0,0.0,0.0,0.0,17.0,11.0,0.0,0.0,13.0,7.0,1.0,1.0,3.0,3.0,8.0,5.0,0.0,0.0,9.0,6.0
5204,Denver,USA,57.0,1993-11-12,1631,1646,UFC 1,1.0,America/Denver,8,Полутяжелый вес,['SUB'],1646.0,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,12.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5205,Denver,USA,138.0,1993-11-12,1646,1924,UFC 1,1.0,America/Denver,7,Средний вес,['SUB'],1646.0,NaN,NaN,7.0,4.0,1.0,1.0,0.0,0.0,3.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Добавляем данные о бойцах в датафрейм с боями

In [31]:
fighter_data_cols = fighters_df.drop(
    columns=["weightCategory.id", "weightCategory.name"]  # add "name" later
).columns
events_df = events_df.join(
    fighters_df[fighter_data_cols].add_prefix("f1_"), on="fighterId_1"
)
events_df = events_df.join(
    fighters_df[fighter_data_cols].add_prefix("f2_"), on="fighterId_2"
)
events_df.head(5)

ValueError: ValueError: columns overlap but no suffix specified: Index(['f1_name', 'f1_weight', 'f1_height', 'f1_armSpan', 'f1_legSwing',
       'f1_dateOfBirth', 'f1_country', 'f1_city', 'f1_timezone'],
      dtype='object')

### Добавляем признак `age`
`age` - количество полных лет бойца на момент боя

In [32]:
def add_age(row: pd.Series) -> pd.Series:
    """
    Add age for both fighters.
    :param row: Row of the events dataframe.
    :return: pd.Series with age of fighters in years.
    """
    result = []
    for prefix in ["f1_", "f2_"]:
        try:
            age = row["eventDate.date"].year - row[prefix + "dateOfBirth"].year
        except Exception:
            age = np.nan
        result.append(age)
    return pd.Series(result)

In [34]:
events_df[["f1_age", "f2_age"]] = events_df[
    ["eventDate.date", "f1_dateOfBirth", "f2_dateOfBirth"]
].apply(lambda row: add_age(row), axis=1)
events_df.head(5)

,city,country,duration,eventDate.date,fighterId_1,fighterId_2,name,rounds,timezone,weightCategory.id,weightCategory.name,winMethods,winnerId,f1_odds,f2_odds,f1_hitsTotal,f1_hitsSuccessful,f1_takedownTotal,f1_takedownSuccessful,f1_submissionAttempts,f1_takeovers,f1_accentedHitsTotal,f1_accentedHitsSuccessful,f1_knockdowns,f1_protectionPassage,f1_hitsHeadTotal,f1_hitsHeadSuccessful,f1_hitsBodyTotal,f1_hitsBodySuccessful,f1_hitsLegsTotal,f1_hitsLegsSuccessful,f1_accentedHitsPositionDistanceTotal,f1_accentedHitsPositionDistanceSuccessful,f1_accentedHitsPositionClinchTotal,f1_accentedHitsPositionClinchSuccessful,f1_accentedHitsPositionParterTotal,f1_accentedHitsPositionParterSuccessful,f2_hitsTotal,f2_hitsSuccessful,f2_takedownTotal,f2_takedownSuccessful,f2_submissionAttempts,f2_takeovers,f2_accentedHitsTotal,f2_accentedHitsSuccessful,f2_knockdowns,f2_protectionPassage,f2_hitsHeadTotal,f2_hitsHeadSuccessful,f2_hitsBodyTotal,f2_hitsBodySuccessful,f2_hitsLegsTotal,f2_hitsLegsSuccessful,f2_accentedHitsPositionDistanceTotal,f2_accentedHitsPositionDistanceSuccessful,f2_accentedHitsPositionClinchTotal,f2_accentedHitsPositionClinchSuccessful,f2_accentedHitsPositionParterTotal,f2_accentedHitsPositionParterSuccessful,f1_name,f1_weight,f1_height,f1_armSpan,f1_legSwing,f1_dateOfBirth,f1_country,f1_city,f1_timezone,f2_name,f2_weight,f2_height,f2_armSpan,f2_legSwing,f2_dateOfBirth,f2_country,f2_city,f2_timezone,f1_age,f2_age
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5201,Denver,USA,104.0,1993-11-12,1646,1923,UFC 1,1.0,America/Denver,7,Средний вес,['SUB'],1646.0,NaN,NaN,4.0,3.0,3.0,1.0,1.0,0.0,2.0,1.0,0.0,2.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
5202,Denver,USA,52.0,1993-11-12,1777,1883,UFC 1,1.0,America/Denver,8,Полутяжелый вес,['SUB'],1777.0,NaN,NaN,9.0,3.0,2.0,1.0,1.0,0.0,9.0,3.0,0.0,1.0,7.0,3.0,1.0,0.0,1.0,0.0,9.0,3.0,0.0,0.0,0.0,0.0,7.0,1.0,0.0,0.0,0.0,0.0,7.0,1.0,0.0,0.0,5.0,0.0,1.0,0.0,1.0,1.0,7.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
5203,Denver,USA,59.0,1993-11-12,1908,1923,UFC 1,1.0,America/Denver,9,Тяжелый вес,['KO'],1923.0,NaN,NaN,3.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,17.0,11.0,0.0,0.0,0.0,0.0,17.0,11.0,0.0,0.0,13.0,7.0,1.0,1.0,3.0,3.0,8.0,5.0,0.0,0.0,9.0,6.0,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
5204,Denver,USA,57.0,1993-11-12,1631,1646,UFC 1,1.0,America/Denver,8,Полутяжелый вес,['SUB'],1646.0,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,12.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
5205,Denver,USA,138.0,1993-11-12,1646,1924,UFC 1,1.0,America/Denver,7,Средний вес,['SUB'],1646.0,NaN,NaN,7.0,4.0,1.0,1.0,0.0,0.0,3.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN


### Добавляем признаки `isHomeCity`, `isHomeCountry`, `isHomeTimezone`
Возможные значения переменных: 0 и 1 \
`isHomeCity` - боец дерется в родном городе \
`isHomeCountry` - боец дерется в родной стране \
`isHomeTimezone` - боец дерется в своем часовом поясе

In [35]:
def get_territorial_cols() -> List[str]:
    """
    Get list of territorial column names for each fighter.
    :return: List of column names with 'f1_' prefix
    for the first fighter and 'f2_' prefix for the second.
    """
    result = []
    for prefix in ["f1_", "f2_"]:
        for key in ["isHomeCity", "isHomeCountry", "isHomeTimezone"]:
            result.append(prefix + key)
    return result


def fill_territorial_cols(row: pd.Series) -> pd.Series:
    """
    Add binary features 'isHomeCity', 'isHomeCountry', 'isHomeTimezone'
    for each fighter.
    :param row: Row of the events dataframe.
    :return: pd.Series with features for both fighters.
    """
    result = []
    for prefix in ["f1_", "f2_"]:
        for key in ["city", "country", "timezone"]:
            result.append(int(row[key] == row[prefix + key]))
    return pd.Series(result)

In [36]:
events_df[get_territorial_cols()] = events_df.apply(
    lambda row: fill_territorial_cols(row), axis=1
)
events_df.head(5)

,city,country,duration,eventDate.date,fighterId_1,fighterId_2,name,rounds,timezone,weightCategory.id,weightCategory.name,winMethods,winnerId,f1_odds,f2_odds,f1_hitsTotal,f1_hitsSuccessful,f1_takedownTotal,f1_takedownSuccessful,f1_submissionAttempts,f1_takeovers,f1_accentedHitsTotal,f1_accentedHitsSuccessful,f1_knockdowns,f1_protectionPassage,f1_hitsHeadTotal,f1_hitsHeadSuccessful,f1_hitsBodyTotal,f1_hitsBodySuccessful,f1_hitsLegsTotal,f1_hitsLegsSuccessful,f1_accentedHitsPositionDistanceTotal,f1_accentedHitsPositionDistanceSuccessful,f1_accentedHitsPositionClinchTotal,f1_accentedHitsPositionClinchSuccessful,f1_accentedHitsPositionParterTotal,f1_accentedHitsPositionParterSuccessful,f2_hitsTotal,f2_hitsSuccessful,f2_takedownTotal,f2_takedownSuccessful,f2_submissionAttempts,f2_takeovers,f2_accentedHitsTotal,f2_accentedHitsSuccessful,f2_knockdowns,f2_protectionPassage,f2_hitsHeadTotal,f2_hitsHeadSuccessful,f2_hitsBodyTotal,f2_hitsBodySuccessful,f2_hitsLegsTotal,f2_hitsLegsSuccessful,f2_accentedHitsPositionDistanceTotal,f2_accentedHitsPositionDistanceSuccessful,f2_accentedHitsPositionClinchTotal,f2_accentedHitsPositionClinchSuccessful,f2_accentedHitsPositionParterTotal,f2_accentedHitsPositionParterSuccessful,f1_name,f1_weight,f1_height,f1_armSpan,f1_legSwing,f1_dateOfBirth,f1_country,f1_city,f1_timezone,f2_name,f2_weight,f2_height,f2_armSpan,f2_legSwing,f2_dateOfBirth,f2_country,f2_city,f2_timezone,f1_age,f2_age,f1_isHomeCity,f1_isHomeCountry,f1_isHomeTimezone,f2_isHomeCity,f2_isHomeCountry,f2_isHomeTimezone
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5201,Denver,USA,104.0,1993-11-12,1646,1923,UFC 1,1.0,America/Denver,7,Средний вес,['SUB'],1646.0,NaN,NaN,4.0,3.0,3.0,1.0,1.0,0.0,2.0,1.0,0.0,2.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0
5202,Denver,USA,52.0,1993-11-12,1777,1883,UFC 1,1.0,America/Denver,8,Полутяжелый вес,['SUB'],1777.0,NaN,NaN,9.0,3.0,2.0,1.0,1.0,0.0,9.0,3.0,0.0,1.0,7.0,3.0,1.0,0.0,1.0,0.0,9.0,3.0,0.0,0.0,0.0,0.0,7.0,1.0,0.0,0.0,0.0,0.0,7.0,1.0,0.0,0.0,5.0,0.0,1.0,0.0,1.0,1.0,7.0,1.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0
5203,Denver,USA,59.0,1993-11-12,1908,1923,UFC 1,1.0,America/Denver,9,Тяжелый вес,['KO'],1923.0,NaN,NaN,3.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,17.0,11.0,0.0,0.0,0.0,0.0,17.0,11.0,0.0,0.0,13.0,7.0,1.0,1.0,3.0,3.0,8.0,5.0,0.0,0.0,9.0,6.0,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0
5204,Denver,USA,57.0,1993-11-12,1631,1646,UFC 1,1.0,America/Denver,8,Полутяжелый вес,['SUB'],1646.0,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,12.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0
5205,Denver,USA,138.0,1993-11-12,1646,1924,UFC 1,1.0,America/Denver,7,Средний вес,['SUB'],1646.0,NaN,NaN,7.0,4.0,1.0,1.0,0.0,0.0,3.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0


### **Какую статистику необходимо посчитать накопительным итогом к бою:**
- **winning_streak** - сумма побед **подряд** по одному бойцу (серия побед) count of winnerId in sequence group by winnerId
- **wins_by_knockowt** - сумма побед нокаутами  count winMethods == ['KO']  group by winnerId
* **wins_by_submissions** - сумма чистых побед (болевой прием, который приводит к сдаче соперника) count winMethods == ['SUB'] group by winnerId
* **striking_accuracy** - точность ударов sum(f1_accentedHitsSuccessful)/sum(f1_accentedHitsTotal) or sum(f2_accentedHitsSuccessful)/sum(f2_accentedHitsTotal)  group by fighterId_1 or fighterId_2
* **Strikes_Landed** - нанесено акцентовых ударов sum(f1_accentedHitsSuccessful) or sum(f2_accentedHitsSuccessful) group by fighterId_1 or fighterId_2
* **Strikes_Attempted** - выброшено акцентовых ударов sum(f1_accentedHitsTotal) or sum(f2_accentedHitsTotal) group by fighterId_1 or fighterId_2
* **grappling_accuracy** - статистика в борьбе sum(f1_takedownSuccessful)/sum(f1_takedownTotal) or sum(f2_takedownSuccessful)/sum(f2_takedownTotal)  group by fighterId_1 or fighterId_2
* **takwdowns_landed** - Тейкдаунов выполнено sum(f1_takedownSuccessful) or sum(f2_takedownSuccessful) group by fighterId_1 or fighterId_2
* **Takedowns Attempted** - попыток Тейкдаунов sum(f1_takedownTotal) or sum(f2_takedownTotal) group by fighterId_1 or fighterId_2
* **Knockdown_ratio** - НОКДАУНОВ ЗА БОЙ/СРЕД. sum(f1_knockdowns)/count of figthts or sum(f2_knockdowns)/count of figthts group by fighterId_1 or fighterId_2
* **AVG_fight_time** - СРЕДНЕЕ ВРЕМЯ БОЯ sum(duration)/count of figthts group by fighterId_1 or fighterId_2
* **KO\TKO** - ко \ все победы
* **кол-во акц ударов успешные \(duration \ (60))**

### Подготовка датафрэйма для кумулятивной суммы по статистике бойцов

In [38]:
stats_events_summary = events_df.copy().reset_index()
stats_events_summary['winner_1'] = stats_events_summary['winnerId'] == stats_events_summary['fighterId_1'] 
stats_events_summary['winner_2'] = stats_events_summary['winnerId'] == stats_events_summary['fighterId_2'] 
stats_events_summary = pd.get_dummies(stats_events_summary, columns = ['winMethods'])
stats_events_summary.columns = stats_events_summary.columns.str.replace('\'','')
fighter1_events = stats_events_summary[['id','eventDate.date','fighterId_1','duration','winner_1','f1_hitsTotal', 'f1_hitsSuccessful', 'f1_takedownTotal',
       'f1_takedownSuccessful', 'f1_submissionAttempts', 'f1_takeovers',
       'f1_accentedHitsTotal', 'f1_accentedHitsSuccessful', 'f1_knockdowns',
       'f1_protectionPassage', 'f1_hitsHeadTotal', 'f1_hitsHeadSuccessful',
       'f1_hitsBodyTotal', 'f1_hitsBodySuccessful', 'f1_hitsLegsTotal',
       'f1_hitsLegsSuccessful', 'f1_accentedHitsPositionDistanceTotal',
       'f1_accentedHitsPositionDistanceSuccessful',
       'f1_accentedHitsPositionClinchTotal',
       'f1_accentedHitsPositionClinchSuccessful',
       'f1_accentedHitsPositionParterTotal',
       'f1_accentedHitsPositionParterSuccessful','winMethods_[DEC]', 'winMethods_[DQ]', 'winMethods_[KO]',
       'winMethods_[NC]', 'winMethods_[SUB]']]
fighter1_events[['fighter_nbr']] = 1 # add what order was in event df
fighter2_events = stats_events_summary[['id','eventDate.date','fighterId_2','duration','winner_2','f2_hitsTotal',
       'f2_hitsSuccessful', 'f2_takedownTotal', 'f2_takedownSuccessful',
       'f2_submissionAttempts', 'f2_takeovers', 'f2_accentedHitsTotal',
       'f2_accentedHitsSuccessful', 'f2_knockdowns', 'f2_protectionPassage',
       'f2_hitsHeadTotal', 'f2_hitsHeadSuccessful', 'f2_hitsBodyTotal',
       'f2_hitsBodySuccessful', 'f2_hitsLegsTotal', 'f2_hitsLegsSuccessful',
       'f2_accentedHitsPositionDistanceTotal',
       'f2_accentedHitsPositionDistanceSuccessful',
       'f2_accentedHitsPositionClinchTotal',
       'f2_accentedHitsPositionClinchSuccessful',
       'f2_accentedHitsPositionParterTotal',
       'f2_accentedHitsPositionParterSuccessful','winMethods_[DEC]', 'winMethods_[DQ]', 'winMethods_[KO]',
       'winMethods_[NC]', 'winMethods_[SUB]']]
fighter2_events[['fighter_nbr']] = 2 # add what order was in event df
col_name =  ['id','eventDate.date','fighterId','duration','winner','_hitsTotal',
    '_hitsSuccessful', '_takedownTotal', '_takedownSuccessful',
       '_submissionAttempts', '_takeovers', '_accentedHitsTotal',
       '_accentedHitsSuccessful', '_knockdowns', '_protectionPassage',
       '_hitsHeadTotal', '_hitsHeadSuccessful', '_hitsBodyTotal',
       '_hitsBodySuccessful', '_hitsLegsTotal', '_hitsLegsSuccessful',
       '_accentedHitsPositionDistanceTotal',
       '_accentedHitsPositionDistanceSuccessful',
       '_accentedHitsPositionClinchTotal',
       '_accentedHitsPositionClinchSuccessful',
       '_accentedHitsPositionParterTotal',
       '_accentedHitsPositionParterSuccessful','fighter_nbr','winMethods_[DEC]', 'winMethods_[DQ]', 'winMethods_[KO]',
       'winMethods_[NC]', 'winMethods_[SUB]']
fighter1_events.columns = col_name
fighter2_events.columns = col_name
f_stats_events_summ = pd.concat([fighter1_events,fighter2_events])
f_stats_events_summ.sort_values(by = ['fighterId','eventDate.date'], axis=0, inplace = True) # df with all firters ordered by ('fighterId','eventDate.date')

In [39]:
def add_cumulative_sum (df:pd.DataFrame, columns:List[str]) -> pd.DataFrame:
    """
    Add cumulative sum for previous fights for input columns list to input df.
    :param df: input DF (should be sorted by fighter and date)
    :param columns: Column names of the dataframe.
    :return: pd.DataFrame with cumulative sum for previous fights for input columns list.
    """
    for column in columns:
        col_name = 'prev_cumsum' + column  
        df[col_name] = df.groupby('fighterId')[column].cumsum() - f_stats_events_summ[column]
    return df

In [40]:
f_stats_events_summ = add_cumulative_sum (f_stats_events_summ, col_name[3:-1]) # df c накопленной суммой

In [41]:
# winning_streak - сумма побед подряд по одному бойцу (серия побед) count of winnerId in sequence group by winnerId
#f_stats_events_summ['winning_streak'] = ?

#wins_by_knockowt 
f_stats_events_summ['wins_by_knockout'] =f_stats_events_summ[(f_stats_events_summ['winner']==True)].groupby('fighterId')['winMethods_[KO]'].cumsum()
f_stats_events_summ.sort_values(by = ['fighterId','eventDate.date'], inplace = True)
f_stats_events_summ['wins_by_knockout'] = f_stats_events_summ['wins_by_knockout'].shift().fillna(0)

In [42]:
#wins_by_submissions
f_stats_events_summ['wins_by_submissions'] =f_stats_events_summ[(f_stats_events_summ['winner']==True)].groupby('fighterId')['winMethods_[SUB]'].cumsum()
f_stats_events_summ.sort_values(by = ['fighterId','eventDate.date'], inplace = True)
f_stats_events_summ['wins_by_submissions'] = f_stats_events_summ['wins_by_submissions'].shift().fillna(0)

In [43]:
f_stats_events_summ.head()

,id,eventDate.date,fighterId,duration,winner,_hitsTotal,_hitsSuccessful,_takedownTotal,_takedownSuccessful,_submissionAttempts,_takeovers,_accentedHitsTotal,_accentedHitsSuccessful,_knockdowns,_protectionPassage,_hitsHeadTotal,_hitsHeadSuccessful,_hitsBodyTotal,_hitsBodySuccessful,_hitsLegsTotal,_hitsLegsSuccessful,_accentedHitsPositionDistanceTotal,_accentedHitsPositionDistanceSuccessful,_accentedHitsPositionClinchTotal,_accentedHitsPositionClinchSuccessful,_accentedHitsPositionParterTotal,_accentedHitsPositionParterSuccessful,fighter_nbr,winMethods_[DEC],winMethods_[DQ],winMethods_[KO],winMethods_[NC],winMethods_[SUB],prev_cumsumduration,prev_cumsumwinner,prev_cumsum_hitsTotal,prev_cumsum_hitsSuccessful,prev_cumsum_takedownTotal,prev_cumsum_takedownSuccessful,prev_cumsum_submissionAttempts,prev_cumsum_takeovers,prev_cumsum_accentedHitsTotal,prev_cumsum_accentedHitsSuccessful,prev_cumsum_knockdowns,prev_cumsum_protectionPassage,prev_cumsum_hitsHeadTotal,prev_cumsum_hitsHeadSuccessful,prev_cumsum_hitsBodyTotal,prev_cumsum_hitsBodySuccessful,prev_cumsum_hitsLegsTotal,prev_cumsum_hitsLegsSuccessful,prev_cumsum_accentedHitsPositionDistanceTotal,prev_cumsum_accentedHitsPositionDistanceSuccessful,prev_cumsum_accentedHitsPositionClinchTotal,prev_cumsum_accentedHitsPositionClinchSuccessful,prev_cumsum_accentedHitsPositionParterTotal,prev_cumsum_accentedHitsPositionParterSuccessful,prev_cumsumfighter_nbr,prev_cumsumwinMethods_[DEC],prev_cumsumwinMethods_[DQ],prev_cumsumwinMethods_[KO],prev_cumsumwinMethods_[NC],wins_by_knockout,wins_by_submissions
6323,5294,2019-10-19 04:00:00,1,300.0,True,51.0,35.0,0.0,0.0,0.0,0.0,51.0,35.0,0.0,0.0,25.0,11.0,15.0,15.0,11.0,9.0,51.0,35.0,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0
6413,39760,2019-12-21 00:00:00,1,300.0,False,37.0,13.0,0.0,0.0,0.0,0.0,37.0,13.0,0.0,0.0,30.0,6.0,3.0,3.0,4.0,4.0,37.0,13.0,0.0,0.0,0.0,0.0,0,0,0,0,0,1,300.0,1,51.0,35.0,0.0,0.0,0.0,0.0,51.0,35.0,0.0,0.0,25.0,11.0,15.0,15.0,11.0,9.0,51.0,35.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,1.0
6596,40040,2020-06-27 00:00:00,1,161.0,True,22.0,15.0,0.0,0.0,0.0,0.0,22.0,15.0,0.0,1.0,12.0,6.0,2.0,2.0,8.0,7.0,21.0,14.0,0.0,0.0,1.0,1.0,0,0,1,0,0,1,600.0,1,88.0,48.0,0.0,0.0,0.0,0.0,88.0,48.0,0.0,0.0,55.0,17.0,18.0,18.0,15.0,13.0,88.0,48.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,4.0
6650,40187,2020-07-25 00:00:00,1,156.0,True,36.0,28.0,0.0,0.0,0.0,0.0,36.0,28.0,0.0,0.0,20.0,12.0,6.0,6.0,10.0,10.0,24.0,19.0,0.0,0.0,12.0,9.0,0,0,1,0,0,1,761.0,2,110.0,63.0,0.0,0.0,0.0,0.0,110.0,63.0,0.0,1.0,67.0,23.0,20.0,20.0,23.0,20.0,109.0,62.0,0.0,0.0,1.0,1.0,0,0,1,0,0,0.0,2.0
6805,40298,2020-11-07 00:00:00,1,300.0,False,55.0,29.0,0.0,0.0,0.0,0.0,55.0,29.0,0.0,0.0,30.0,7.0,3.0,1.0,22.0,21.0,54.0,29.0,1.0,0.0,0.0,0.0,1,0,0,0,0,1,917.0,3,146.0,91.0,0.0,0.0,0.0,0.0,146.0,91.0,0.0,1.0,87.0,35.0,26.0,26.0,33.0,30.0,133.0,81.0,0.0,0.0,13.0,10.0,0,0,2,0,0,0.0,3.0


In [44]:
 f_stats_events_summ['count_of_fights'] = f_stats_events_summ.groupby('fighterId')['fighter_nbr'].cumcount()

In [45]:

#striking_accuracy 
f_stats_events_summ['striking_accuracy']  = f_stats_events_summ['prev_cumsum_accentedHitsSuccessful']/f_stats_events_summ['prev_cumsum_accentedHitsTotal']
#Strikes_Landed == f_stats_events_summ['prev_cumsum_accentedHitsSuccessful']

#Strikes_Attempted == f_stats_events_summ['prev_cumsum_accentedHitsTotal']

#grappling_accuracy 
f_stats_events_summ['grappling_accuracy'] = f_stats_events_summ['prev_cumsum_takedownSuccessful']/f_stats_events_summ['prev_cumsum_takedownTotal']

#takwdowns_landed  == f_stats_events_summ['prev_cumsum_takedownTotall']
#Takedowns Attempted == f_stats_events_summ['prev_cumsum_takedownSuccessful']
#Knockdown_ratio 
f_stats_events_summ['Knockdown_ratio'] = f_stats_events_summ['prev_cumsum_knockdowns'] /f_stats_events_summ['count_of_fights']
#AVG_fight_time
f_stats_events_summ['AVG_fight_time'] = f_stats_events_summ['prev_cumsumduration']/f_stats_events_summ['count_of_fights']

#KO/TKO - ко \ все победы

# кол-во акц ударов успешные \ (duration (60))
f_stats_events_summ['stricing_sucss_per_duration']  = f_stats_events_summ['prev_cumsum_accentedHitsSuccessful'] * 60 /f_stats_events_summ['prev_cumsumduration']

In [46]:
f_stats_events_summ['striking_accuracy'] = f_stats_events_summ['striking_accuracy'].fillna(0).round(0)
f_stats_events_summ['grappling_accuracy'] = f_stats_events_summ['grappling_accuracy'].fillna(0).round(0)
f_stats_events_summ['stricing_sucss_per_duration'] = f_stats_events_summ['stricing_sucss_per_duration'].fillna(0).round(0)
f_stats_events_summ['AVG_fight_time'] = f_stats_events_summ['stricing_sucss_per_duration'].fillna(0).round(0)
f_stats_events_summ['Knockdown_ratio'] = f_stats_events_summ['stricing_sucss_per_duration'].fillna(0).round(0)

In [47]:
events_df = events_df.join(
    f_stats_events_summ[f_stats_events_summ['fighter_nbr'] == 0].add_prefix("f1_"), on="id")
events_df = events_df.join(
    f_stats_events_summ[f_stats_events_summ['fighter_nbr'] == 1].add_prefix("f2_"), on="id")
events_df.head(5)

,city,country,duration,eventDate.date,fighterId_1,fighterId_2,name,rounds,timezone,weightCategory.id,weightCategory.name,winMethods,winnerId,f1_odds,f2_odds,f1_hitsTotal,f1_hitsSuccessful,f1_takedownTotal,f1_takedownSuccessful,f1_submissionAttempts,f1_takeovers,f1_accentedHitsTotal,f1_accentedHitsSuccessful,f1_knockdowns,f1_protectionPassage,f1_hitsHeadTotal,f1_hitsHeadSuccessful,f1_hitsBodyTotal,f1_hitsBodySuccessful,f1_hitsLegsTotal,f1_hitsLegsSuccessful,f1_accentedHitsPositionDistanceTotal,f1_accentedHitsPositionDistanceSuccessful,f1_accentedHitsPositionClinchTotal,f1_accentedHitsPositionClinchSuccessful,f1_accentedHitsPositionParterTotal,f1_accentedHitsPositionParterSuccessful,f2_hitsTotal,f2_hitsSuccessful,f2_takedownTotal,f2_takedownSuccessful,f2_submissionAttempts,f2_takeovers,f2_accentedHitsTotal,f2_accentedHitsSuccessful,f2_knockdowns,f2_protectionPassage,f2_hitsHeadTotal,f2_hitsHeadSuccessful,f2_hitsBodyTotal,f2_hitsBodySuccessful,f2_hitsLegsTotal,f2_hitsLegsSuccessful,f2_accentedHitsPositionDistanceTotal,f2_accentedHitsPositionDistanceSuccessful,f2_accentedHitsPositionClinchTotal,f2_accentedHitsPositionClinchSuccessful,f2_accentedHitsPositionParterTotal,f2_accentedHitsPositionParterSuccessful,f1_name,f1_weight,f1_height,f1_armSpan,f1_legSwing,f1_dateOfBirth,f1_country,f1_city,f1_timezone,f2_name,f2_weight,f2_height,f2_armSpan,f2_legSwing,f2_dateOfBirth,f2_country,f2_city,f2_timezone,f1_age,f2_age,f1_isHomeCity,f1_isHomeCountry,f1_isHomeTimezone,f2_isHomeCity,f2_isHomeCountry,f2_isHomeTimezone,f1_id,f1_eventDate.date,f1_fighterId,f1_duration,f1_winner,f1__hitsTotal,f1__hitsSuccessful,f1__takedownTotal,f1__takedownSuccessful,f1__submissionAttempts,f1__takeovers,f1__accentedHitsTotal,f1__accentedHitsSuccessful,f1__knockdowns,f1__protectionPassage,f1__hitsHeadTotal,f1__hitsHeadSuccessful,f1__hitsBodyTotal,f1__hitsBodySuccessful,f1__hitsLegsTotal,f1__hitsLegsSuccessful,f1__accentedHitsPositionDistanceTotal,f1__accentedHitsPositionDistanceSuccessful,f1__accentedHitsPositionClinchTotal,f1__accentedHitsPositionClinchSuccessful,f1__accentedHitsPositionParterTotal,f1__accentedHitsPositionParterSuccessful,f1_fighter_nbr,f1_winMethods_[DEC],f1_winMethods_[DQ],f1_winMethods_[KO],f1_winMethods_[NC],f1_winMethods_[SUB],f1_prev_cumsumduration,f1_prev_cumsumwinner,f1_prev_cumsum_hitsTotal,f1_prev_cumsum_hitsSuccessful,f1_prev_cumsum_takedownTotal,f1_prev_cumsum_takedownSuccessful,f1_prev_cumsum_submissionAttempts,f1_prev_cumsum_takeovers,f1_prev_cumsum_accentedHitsTotal,f1_prev_cumsum_accentedHitsSuccessful,f1_prev_cumsum_knockdowns,f1_prev_cumsum_protectionPassage,f1_prev_cumsum_hitsHeadTotal,f1_prev_cumsum_hitsHeadSuccessful,f1_prev_cumsum_hitsBodyTotal,f1_prev_cumsum_hitsBodySuccessful,f1_prev_cumsum_hitsLegsTotal,f1_prev_cumsum_hitsLegsSuccessful,f1_prev_cumsum_accentedHitsPositionDistanceTotal,f1_prev_cumsum_accentedHitsPositionDistanceSuccessful,f1_prev_cumsum_accentedHitsPositionClinchTotal,f1_prev_cumsum_accentedHitsPositionClinchSuccessful,f1_prev_cumsum_accentedHitsPositionParterTotal,f1_prev_cumsum_accentedHitsPositionParterSuccessful,f1_prev_cumsumfighter_nbr,f1_prev_cumsumwinMethods_[DEC],f1_prev_cumsumwinMethods_[DQ],f1_prev_cumsumwinMethods_[KO],f1_prev_cumsumwinMethods_[NC],f1_wins_by_knockout,f1_wins_by_submissions,f1_count_of_fights,f1_striking_accuracy,f1_grappling_accuracy,f1_Knockdown_ratio,f1_AVG_fight_time,f1_stricing_sucss_per_duration,f2_id,f2_eventDate.date,f2_fighterId,f2_duration,f2_winner,f2__hitsTotal,f2__hitsSuccessful,f2__takedownTotal,f2__takedownSuccessful,f2__submissionAttempts,f2__takeovers,f2__accentedHitsTotal,f2__accentedHitsSuccessful,f2__knockdowns,f2__protectionPassage,f2__hitsHeadTotal,f2__hitsHeadSuccessful,f2__hitsBodyTotal,f2__hitsBodySuccessful,f2__hitsLegsTotal,f2__hitsLegsSuccessful,f2__accentedHitsPositionDistanceTotal,f2__accentedHitsPositionDistanceSuccessful,f2__accentedHitsPositionClinchTotal,f2__accentedHitsPositionClinchSuccessful,f2__accentedHitsPositionParterTotal,f2__accente

In [48]:
f_stats_events_summ.head()

,id,eventDate.date,fighterId,duration,winner,_hitsTotal,_hitsSuccessful,_takedownTotal,_takedownSuccessful,_submissionAttempts,_takeovers,_accentedHitsTotal,_accentedHitsSuccessful,_knockdowns,_protectionPassage,_hitsHeadTotal,_hitsHeadSuccessful,_hitsBodyTotal,_hitsBodySuccessful,_hitsLegsTotal,_hitsLegsSuccessful,_accentedHitsPositionDistanceTotal,_accentedHitsPositionDistanceSuccessful,_accentedHitsPositionClinchTotal,_accentedHitsPositionClinchSuccessful,_accentedHitsPositionParterTotal,_accentedHitsPositionParterSuccessful,fighter_nbr,winMethods_[DEC],winMethods_[DQ],winMethods_[KO],winMethods_[NC],winMethods_[SUB],prev_cumsumduration,prev_cumsumwinner,prev_cumsum_hitsTotal,prev_cumsum_hitsSuccessful,prev_cumsum_takedownTotal,prev_cumsum_takedownSuccessful,prev_cumsum_submissionAttempts,prev_cumsum_takeovers,prev_cumsum_accentedHitsTotal,prev_cumsum_accentedHitsSuccessful,prev_cumsum_knockdowns,prev_cumsum_protectionPassage,prev_cumsum_hitsHeadTotal,prev_cumsum_hitsHeadSuccessful,prev_cumsum_hitsBodyTotal,prev_cumsum_hitsBodySuccessful,prev_cumsum_hitsLegsTotal,prev_cumsum_hitsLegsSuccessful,prev_cumsum_accentedHitsPositionDistanceTotal,prev_cumsum_accentedHitsPositionDistanceSuccessful,prev_cumsum_accentedHitsPositionClinchTotal,prev_cumsum_accentedHitsPositionClinchSuccessful,prev_cumsum_accentedHitsPositionParterTotal,prev_cumsum_accentedHitsPositionParterSuccessful,prev_cumsumfighter_nbr,prev_cumsumwinMethods_[DEC],prev_cumsumwinMethods_[DQ],prev_cumsumwinMethods_[KO],prev_cumsumwinMethods_[NC],wins_by_knockout,wins_by_submissions,count_of_fights,striking_accuracy,grappling_accuracy,Knockdown_ratio,AVG_fight_time,stricing_sucss_per_duration
6323,5294,2019-10-19 04:00:00,1,300.0,True,51.0,35.0,0.0,0.0,0.0,0.0,51.0,35.0,0.0,0.0,25.0,11.0,15.0,15.0,11.0,9.0,51.0,35.0,0.0,0.0,0.0,0.0,0,0,0,0,0,1,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0
6413,39760,2019-12-21 00:00:00,1,300.0,False,37.0,13.0,0.0,0.0,0.0,0.0,37.0,13.0,0.0,0.0,30.0,6.0,3.0,3.0,4.0,4.0,37.0,13.0,0.0,0.0,0.0,0.0,0,0,0,0,0,1,300.0,1,51.0,35.0,0.0,0.0,0.0,0.0,51.0,35.0,0.0,0.0,25.0,11.0,15.0,15.0,11.0,9.0,51.0,35.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,1.0,1,1.0,0.0,7.0,7.0,7.0
6596,40040,2020-06-27 00:00:00,1,161.0,True,22.0,15.0,0.0,0.0,0.0,0.0,22.0,15.0,0.0,1.0,12.0,6.0,2.0,2.0,8.0,7.0,21.0,14.0,0.0,0.0,1.0,1.0,0,0,1,0,0,1,600.0,1,88.0,48.0,0.0,0.0,0.0,0.0,88.0,48.0,0.0,0.0,55.0,17.0,18.0,18.0,15.0,13.0,88.0,48.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0.0,4.0,2,1.0,0.0,5.0,5.0,5.0
6650,40187,2020-07-25 00:00:00,1,156.0,True,36.0,28.0,0.0,0.0,0.0,0.0,36.0,28.0,0.0,0.0,20.0,12.0,6.0,6.0,10.0,10.0,24.0,19.0,0.0,0.0,12.0,9.0,0,0,1,0,0,1,761.0,2,110.0,63.0,0.0,0.0,0.0,0.0,110.0,63.0,0.0,1.0,67.0,23.0,20.0,20.0,23.0,20.0,109.0,62.0,0.0,0.0,1.0,1.0,0,0,1,0,0,0.0,2.0,3,1.0,0.0,5.0,5.0,5.0
6805,40298,2020-11-07 00:00:00,1,300.0,False,55.0,29.0,0.0,0.0,0.0,0.0,55.0,29.0,0.0,0.0,30.0,7.0,3.0,1.0,22.0,21.0,54.0,29.0,1.0,0.0,0.0,0.0,1,0,0,0,0,1,917.0,3,146.0,91.0,0.0,0.0,0.0,0.0,146.0,91.0,0.0,1.0,87.0,35.0,26.0,26.0,33.0,30.0,133.0,81.0,0.0,0.0,13.0,10.0,0,0,2,0,0,0.0,3.0,4,1.0,0.0,6.0,6.0,6.0


### Считаем разницу для колонок

In [49]:
df = events_df.copy()

# Колонки, разницу для которых мы считаем
cols = [
 'hitsTotal',
 'hitsSuccessful',
 'takedownTotal',
 'takedownSuccessful',
 'submissionAttempts',
 'takeovers',
 'accentedHitsTotal',
 'accentedHitsSuccessful',
 'knockdowns',
 'protectionPassage',
 'hitsHeadTotal',
 'hitsHeadSuccessful',
 'hitsBodyTotal',
 'hitsBodySuccessful',
 'hitsLegsTotal',
 'hitsLegsSuccessful',
 'accentedHitsPositionDistanceTotal',
 'accentedHitsPositionDistanceSuccessful',
 'accentedHitsPositionClinchTotal',
 'accentedHitsPositionClinchSuccessful',
 'accentedHitsPositionParterTotal',
 'accentedHitsPositionParterSuccessful',
 'weight',
 'height', 
 'armSpan', 
 'legSwing', 
]


def difference(df, cols):
    # цикл заменяет столбцы характеристик каждого бойца столбцами разницы этих характеристик
    for col in cols:
        df[col+'_difference'] = df['f1_'+col] - df['f2_'+col]
        df.drop(columns=['f1_'+col, 'f2_'+col], inplace=True)
        # print(col, 'difference calculated') # Для дебага

    df['age'] = df.f1_age-df.f2_age # не стал удалять столбцы с возрастом

    return df

In [50]:
difference(df, cols)

,city,country,duration,eventDate.date,fighterId_1,fighterId_2,name,rounds,timezone,weightCategory.id,weightCategory.name,winMethods,winnerId,f1_odds,f2_odds,f1_name,f1_dateOfBirth,f1_country,f1_city,f1_timezone,f2_name,f2_dateOfBirth,f2_country,f2_city,f2_timezone,f1_age,f2_age,f1_isHomeCity,f1_isHomeCountry,f1_isHomeTimezone,f2_isHomeCity,f2_isHomeCountry,f2_isHomeTimezone,f1_id,f1_eventDate.date,f1_fighterId,f1_duration,f1_winner,f1__hitsTotal,f1__hitsSuccessful,f1__takedownTotal,f1__takedownSuccessful,f1__submissionAttempts,f1__takeovers,f1__accentedHitsTotal,f1__accentedHitsSuccessful,f1__knockdowns,f1__protectionPassage,f1__hitsHeadTotal,f1__hitsHeadSuccessful,f1__hitsBodyTotal,f1__hitsBodySuccessful,f1__hitsLegsTotal,f1__hitsLegsSuccessful,f1__accentedHitsPositionDistanceTotal,f1__accentedHitsPositionDistanceSuccessful,f1__accentedHitsPositionClinchTotal,f1__accentedHitsPositionClinchSuccessful,f1__accentedHitsPositionParterTotal,f1__accentedHitsPositionParterSuccessful,f1_fighter_nbr,f1_winMethods_[DEC],f1_winMethods_[DQ],f1_winMethods_[KO],f1_winMethods_[NC],f1_winMethods_[SUB],f1_prev_cumsumduration,f1_prev_cumsumwinner,f1_prev_cumsum_hitsTotal,f1_prev_cumsum_hitsSuccessful,f1_prev_cumsum_takedownTotal,f1_prev_cumsum_takedownSuccessful,f1_prev_cumsum_submissionAttempts,f1_prev_cumsum_takeovers,f1_prev_cumsum_accentedHitsTotal,f1_prev_cumsum_accentedHitsSuccessful,f1_prev_cumsum_knockdowns,f1_prev_cumsum_protectionPassage,f1_prev_cumsum_hitsHeadTotal,f1_prev_cumsum_hitsHeadSuccessful,f1_prev_cumsum_hitsBodyTotal,f1_prev_cumsum_hitsBodySuccessful,f1_prev_cumsum_hitsLegsTotal,f1_prev_cumsum_hitsLegsSuccessful,f1_prev_cumsum_accentedHitsPositionDistanceTotal,f1_prev_cumsum_accentedHitsPositionDistanceSuccessful,f1_prev_cumsum_accentedHitsPositionClinchTotal,f1_prev_cumsum_accentedHitsPositionClinchSuccessful,f1_prev_cumsum_accentedHitsPositionParterTotal,f1_prev_cumsum_accentedHitsPositionParterSuccessful,f1_prev_cumsumfighter_nbr,f1_prev_cumsumwinMethods_[DEC],f1_prev_cumsumwinMethods_[DQ],f1_prev_cumsumwinMethods_[KO],f1_prev_cumsumwinMethods_[NC],f1_wins_by_knockout,f1_wins_by_submissions,f1_count_of_fights,f1_striking_accuracy,f1_grappling_accuracy,f1_Knockdown_ratio,f1_AVG_fight_time,f1_stricing_sucss_per_duration,f2_id,f2_eventDate.date,f2_fighterId,f2_duration,f2_winner,f2__hitsTotal,f2__hitsSuccessful,f2__takedownTotal,f2__takedownSuccessful,f2__submissionAttempts,f2__takeovers,f2__accentedHitsTotal,f2__accentedHitsSuccessful,f2__knockdowns,f2__protectionPassage,f2__hitsHeadTotal,f2__hitsHeadSuccessful,f2__hitsBodyTotal,f2__hitsBodySuccessful,f2__hitsLegsTotal,f2__hitsLegsSuccessful,f2__accentedHitsPositionDistanceTotal,f2__accentedHitsPositionDistanceSuccessful,f2__accentedHitsPositionClinchTotal,f2__accentedHitsPositionClinchSuccessful,f2__accentedHitsPositionParterTotal,f2__accentedHitsPositionParterSuccessful,f2_fighter_nbr,f2_winMethods_[DEC],f2_winMethods_[DQ],f2_winMethods_[KO],f2_winMethods_[NC],f2_winMethods_[SUB],f2_prev_cumsumduration,f2_prev_cumsumwinner,f2_prev_cumsum_hitsTotal,f2_prev_cumsum_hitsSuccessful,f2_prev_cumsum_takedownTotal,f2_prev_cumsum_takedownSuccessful,f2_prev_cumsum_submissionAttempts,f2_prev_cumsum_takeovers,f2_prev_cumsum_accentedHitsTotal,f2_prev_cumsum_accentedHitsSuccessful,f2_prev_cumsum_knockdowns,f2_prev_cumsum_protectionPassage,f2_prev_cumsum_hitsHeadTotal,f2_prev_cumsum_hitsHeadSuccessful,f2_prev_cumsum_hitsBodyTotal,f2_prev_cumsum_hitsBodySuccessful,f2_prev_cumsum_hitsLegsTotal,f2_prev_cumsum_hitsLegsSuccessful,f2_prev_cumsum_accentedHitsPositionDistanceTotal,f2_prev_cumsum_accentedHitsPositionDistanceSuccessful,f2_prev_cumsum_accentedHitsPositionClinchTotal,f2_prev_cumsum_accentedHitsPositionClinchSuccessful,f2_prev_cumsum_accentedHitsPositionParterTotal,f2_prev_cumsum_accentedHitsPositionParterSuccessful,f2_prev_cumsumfighter_nbr,f2_prev_cumsumwinMethods_[DEC],f2_prev_cumsumwinMethods_[DQ],f2_prev_cumsumwinMethods_[KO],f2_prev_cumsumwinMethods_[NC],f2_wins_by_knockout,f2

### Корреляционная карта параметров бойцов

In [55]:
import matplotlib.pyplot as plt

f = plt.figure(figsize=(19, 15))
plt.matshow(df.notnull().corr(), fignum=f.number)
plt.xticks(range(df.select_dtypes(['number']).shape[1]), df.select_dtypes(['number']).columns, fontsize=14, rotation=90)
plt.yticks(range(df.select_dtypes(['number']).shape[1]), df.select_dtypes(['number']).columns, fontsize=14)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=14)
plt.title('Корреляционная карта параметров бойцов', fontsize=18);
# plt.savefig('foo.png')

Error in callback <function flush_figures at 0x7fd014e01320> (for post_execute):


In [642]:
cdf = df[['weightCategory.name', 'winMethods', 'age']].groupby(['weightCategory.name', 'winMethods']).count().reset_index()
cdf1 = df[['weightCategory.name', 'age']].groupby(['weightCategory.name']).count().reset_index()
cdf2 = df[['winMethods', 'age']].groupby(['winMethods']).count().reset_index()

In [643]:
import plotly.graph_objs as go

# 1-й уровень, центр диаграммы
labels = ["Всего событий: " + str(sum(cdf.age))]
parents = [""]
values = [sum(cdf.age)]

# 2-й уровень, "промежуточный"
second_level_dict = {x:cdf1['weightCategory.name'][x] + ' ' + str(cdf1.age[x]) for x in cdf1.index}
labels += map(lambda x: second_level_dict[x], cdf1.index)
parents += [labels[0]]*len(cdf1)
values += cdf1.age.tolist()

fig = go.Figure(go.Sunburst(
    labels = labels,
    parents = parents,
    values=values,
    branchvalues="total"
))
fig.update_layout(margin = dict(t=0, l=0, r=0, b=0))

fig.show()

Unsupported

In [634]:
# 1-й уровень, центр диаграммы
labels = ["Всего событий: " + str(sum(cdf.age))]
parents = [""]
values = [sum(cdf.age)]

# 2-й уровень, "промежуточный"
second_level_dict = {x:cdf2['winMethods'][x] + ' ' + str(cdf2.age[x]) for x in cdf2.index}
labels += map(lambda x: second_level_dict[x], cdf2.index)
parents += [labels[0]]*len(cdf2)
values += cdf2.age.tolist()

fig = go.Figure(go.Sunburst(
    labels = labels,
    parents = parents,
    values=values,
    branchvalues="total"
))
fig.update_layout(margin = dict(t=0, l=0, r=0, b=0))

fig.show()

Unsupported